# Nestlé HR Policy Assistant - Course End Project

**Project**: Crafting an AI-Powered HR Assistant for Nestlé's HR Policy Documents

This notebook demonstrates the complete workflow for creating a conversational chatbot that responds to user inquiries using PDF document information.

## Step 1: Import Essential Tools and Set Up OpenAI's API Environment

In [ ]:
# RESTART KERNEL FIRST, then run this cell
import subprocess
import sys

# Uninstall conflicting packages
subprocess.run([sys.executable, '-m', 'pip', 'uninstall', '-y', 'pydantic', 'chromadb', 'gradio', 'fastapi'])

# Install working versions in correct order
subprocess.run([sys.executable, '-m', 'pip', 'install', 'pydantic==1.10.12'])
subprocess.run([sys.executable, '-m', 'pip', 'install', 'chromadb==0.4.15'])
subprocess.run([sys.executable, '-m', 'pip', 'install', 'gradio==3.50.2'])
subprocess.run([sys.executable, '-m', 'pip', 'install', 'langchain', 'langchain-community', 'langchain-openai', 'pypdf', 'openai', 'pysqlite3-binary'])

print("✅ Packages installed. RESTART KERNEL NOW before proceeding.")

In [ ]:
# Fix SQLite BEFORE any other imports
import sys
try:
    import pysqlite3
    sys.modules['sqlite3'] = pysqlite3
except ImportError:
    pass

# Import essential libraries
import os
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_community.vectorstores import Chroma
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA

# Set up OpenAI API key from environment variable
if not os.getenv('OPENAI_API_KEY'):
    raise ValueError('Please set OPENAI_API_KEY environment variable')

print("✅ Essential tools imported and OpenAI API environment set up")

## Step 2: Load Nestlé's HR Policy Using PyPDFLoader and Split for Processing

In [ ]:
# Load PDF document
pdf_path = "1728286846_the_nestle_hr_policy_pdf_2012.pdf"
loader = PyPDFLoader(pdf_path)
documents = loader.load()

# Split documents into chunks
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200
)
texts = text_splitter.split_documents(documents)

print(f"✅ Loaded PDF with {len(documents)} pages")
print(f"✅ Split into {len(texts)} text chunks")

## Step 3: Create Vector Representations Using ChromaDB and OpenAI's Embeddings

In [ ]:
# Create embeddings
embeddings = OpenAIEmbeddings()

# Create vector store using ChromaDB
vectorstore = Chroma.from_documents(
    documents=texts,
    embedding=embeddings
)

print("✅ Vector representations created using ChromaDB and OpenAI embeddings")

## Step 4: Build Question-Answering System Using GPT-3.5 Turbo

In [ ]:
# Initialize GPT-3.5 Turbo model
llm = ChatOpenAI(
    model_name="gpt-3.5-turbo",
    temperature=0.3
)

# Create retriever
retriever = vectorstore.as_retriever(search_kwargs={"k": 3})

print("✅ Question-answering system built with GPT-3.5 Turbo")

## Step 5: Create Prompt Template for Chatbot Understanding

In [ ]:
# Create prompt template
template = """You are a Nestlé HR assistant. Use the following context from Nestlé's HR policy documents to answer the question.

Context: {context}

Question: {question}

Instructions:
- Provide accurate information based on the context
- If information is not available, suggest contacting HR
- Maintain a professional, helpful tone

Answer:"""

prompt = PromptTemplate(
    template=template,
    input_variables=["context", "question"]
)

# Create QA chain
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    chain_type_kwargs={"prompt": prompt}
)

print("✅ Prompt template created for chatbot understanding")

## Step 6: Build User-Friendly Chatbot Interface with Gradio

In [ ]:
def chat_function(message):
    """Process user message and return response"""
    try:
        response = qa_chain.invoke({"query": message})
        return response["result"]
    except Exception as e:
        return f"I apologize, but I'm having trouble accessing the information. Please contact HR directly. Error: {str(e)}"

# Create Gradio interface
import gradio as gr

interface = gr.Interface(
    fn=chat_function,
    inputs=gr.Textbox(placeholder="Ask about HR policies...", label="Your Question", lines=2),
    outputs=gr.Textbox(label="HR Assistant Response", lines=6),
    title="🤖 Nestlé HR Policy Assistant",
    description="Ask questions about Nestlé HR policies and procedures.",
    examples=[
        ["What is Nestlé's approach to performance management?"],
        ["Tell me about career development opportunities"],
        ["What are Nestlé's diversity and inclusion policies?"]
    ],
    allow_flagging="never"
)

print("✅ User-friendly chatbot interface created with Gradio")

## Step 7: Launch the Chatbot

In [ ]:
# Launch the Gradio interface
interface.launch(share=True)

## Testing the System

In [ ]:
# Test the QA system with example questions
test_questions = [
    "What is Nestlé's approach to performance management?",
    "Tell me about career development opportunities",
    "What are Nestlé's diversity and inclusion policies?"
]

print("Testing the HR Assistant:")
print("=" * 80)

for i, question in enumerate(test_questions, 1):
    print(f"\nTest {i}: {question}")
    answer = chat_function(question)
    print(f"Answer: {answer}")
    print("-" * 80)